In [1]:
# project configurations

import sqlalchemy as sa
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime
import os
import zipfile
import uuid
import matplotlib
import seaborn as sns
from os.path import basename
from datetime import datetime, date, time, timedelta


client_pg = sa.create_engine('postgresql://localhost:5432/postgres')

In [6]:
# question number one part two

number_one_part_two = """
    WITH processed_transactions AS (
        (select u.id, ((t.amount / power(10, cd.exponent)) * fx.rate) AS amount, t.created_date, t.type, t.state
         from users u
                  join transactions t on t.user_id = u.id
                  JOIN fx_rates fx ON (fx.ccy = t.currency AND fx.base_ccy = 'USD')
                  JOIN currency_details cd ON cd.currency = t.currency
           and t.currency != 'USD')
        union
        (SELECT u.id, (t.amount / power(10, cd.exponent)) as amount, t.created_date, t.type, t.state
         from users u
                  join transactions t on t.user_id = u.id
                  join currency_details cd ON cd.currency = t.currency
           and t.currency = 'USD')
        order by created_date asc
    )
    select * from
    processed_transactions pt
    where pt.amount > 10
    and pt.type = 'CARD_PAYMENT'
    and pt.state = 'COMPLETED'
    and pt.id IN (select distinct on (pt.id) pt.id from processed_transactions pt);
"""

df_first = pd.read_sql(number_one_part_two, client_pg)
df_first.head()


,id,amount,created_date,type,state
0,488bdbb9-a360-4bd7-81fc-6130990693a3,44.230054,2015-07-11 17:34:06.285,CARD_PAYMENT,COMPLETED
1,55525a77-e67f-458a-8ba8-8c229f91ddb0,36.891376,2015-07-24 06:50:00.598,CARD_PAYMENT,COMPLETED
2,55525a77-e67f-458a-8ba8-8c229f91ddb0,15.970864,2015-07-24 07:56:27.251,CARD_PAYMENT,COMPLETED
3,06bb2d68-bf61-4030-8447-9de64d3ce490,331.250000,2015-07-24 10:58:03.535,CARD_PAYMENT,COMPLETED
4,55525a77-e67f-458a-8ba8-8c229f91ddb0,12.248729,2015-07-25 06:53:03.505,CARD_PAYMENT,COMPLETED
